# Asset Pricing - Final Project

In [1]:
import getFamaFrenchFactors as gff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
import yfinance as yf


In [2]:
plt.style.use('fivethirtyeight')
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': 'Ubuntu',
    'font.monospace': 'Ubuntu Mono',
    'font.size': 14,

    'axes.labelweight': 'normal',
    'axes.labelcolor': 'black',
    'axes.titleweight': 'normal',
    'axes.titlesize': 16,
    'axes.titlelocation': 'center',
    'axes.labelpad': 10,
    'axes.titleweight': 'bold',
    'axes.titlepad': 15,
    
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14,
    
    'figure.titlesize': 18,
    'figure.figsize': (10, 6),
    'figure.autolayout': True,
    'figure.subplot.wspace': 0.25,
    'figure.subplot.wspace': 0.4,

    'lines.linewidth': 2
})

## Data Loading and Preprocessing

### Load Fama-French 3-Factor Model

In [12]:
daily_factors = pd.read_csv(
    'datasets/ff_3factor_daily.csv',
    skiprows=5,
    skipfooter=1,
    names=['date', 'mkt-rf', 'smb', 'hml', 'rf'],
    index_col=[0],
    parse_dates=[0],
    engine='python'
    )/100

monthly_factors = pd.read_csv(
    'datasets/ff_3factor_monthly.csv',
    skiprows=5,
    skipfooter=1,
    names=['date', 'mkt-rf', 'smb', 'hml', 'rf'],
    index_col=[0],
    parse_dates=[0],
    date_parser=lambda x: pd.to_datetime(x, format='%Y%m'),
    engine='python'
    )/100
monthly_factors.index = monthly_factors.index.to_period('M')

annual_factors = pd.read_csv(
    'datasets/ff_3factor_annual.csv',
    skiprows=5,
    skipfooter=1,
    names=['date', 'mkt-rf', 'smb', 'hml', 'rf'],
    index_col=[0],
    parse_dates=[0],
    engine='python'
    )/100
annual_factors.index = annual_factors.index.to_period('Y')

### Load US Stock Market Sector Indexes

In [13]:
indexes = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLK', 'XLU']
start_date = "2004-01-01"
end_date = "2012-01-01"

# load index prices from yahoo finance
daily_prices = yf.download(
    indexes,
    start=start_date,
    )['Close']
daily_prices.index.rename('date', inplace=True)

# calculate daily, monthly, and annual returns
daily_returns = daily_prices.pct_change().dropna()

monthly_returns = daily_prices.resample('MS').last().pct_change().dropna()
monthly_returns.index = monthly_returns.index.to_period('M')

annual_returns = daily_prices.resample('YS').last().pct_change().dropna()
annual_returns.index = annual_returns.index.to_period('A')

[*********************100%***********************]  9 of 9 completed


### Merge dataframes and subtract $R_f$ from index returns

In [22]:
# merge daily factors and index returns
df_daily = pd.merge(
    daily_factors, daily_returns,
    left_index=True, right_index=True,
    how='inner'
    )

# merge monthly factors and index returns
df_monthly = pd.merge(
    monthly_factors, monthly_returns,
    left_index=True, right_index=True,
    how='inner'
    )

# merge annual factors and index returns
df_annual = pd.merge(
    annual_factors, annual_returns,
    left_index=True, right_index=True,
    how='inner'
    )

# subtract risk free rate from index returns
for index in indexes: 
    df_daily[index] = df_daily[index] - df_daily['rf']
    df_monthly[index] = df_monthly[index] - df_monthly['rf']
    df_annual[index] = df_annual[index] - df_annual['rf']